In [84]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
import nltk
import json
import os
import sys
import time
import operator
#import re
import string

In [85]:
########### Set up Output File
outdir = '..//data//clean_data'

##### Contain 2 column : 1. Replaced name of entity / 2. Real name of entity
reverse_entity = open(os.path.join(outdir,os.path.basename('entity_total.csv')),'w') 

#####
entity_replaced = open(os.path.join(outdir,os.path.basename('entity_replaced.csv')),'w',encoding='utf-8')



In [86]:
########### Trigger word
triggers_raw = open('..//data//preprocessing//triggers.txt').read().split('\n')[:-1]
triggers = []
for row in triggers_raw :
    triggers.append(row.split('\t')[-1])

In [87]:
########### Entity sources

#### Drug synonyms from drugbank
drugs = json.load(open('..//data/preprocessing/drug_corpus/drug_name_synonyms.json'))

#### Herb scientific name and its synonyms (not alias)
herbs = json.load(open('..//data/preprocessing/herb_corpus/herb_science_synonyms.json'))

#### standard ref without effect
ref_without = json.load(open('..//data/preprocessing/standard_ref/standard_ref_without_effect.json'))

#### standard ref with effect
ref_effect = open('..//data/preprocessing/standard_ref/standard_ref_effect.csv').read().split('\n')[:-1]

#### Adverse effect corpus
adverse_effect = json.load(open('..//data/preprocessing/adverse_effect_corpus/adverse_effect.json'))





In [88]:
########## Create name for entity

###### trigger entity
trigger_entity = {}
for each in triggers :
    trigger_entity[each] = 'trigger_entity'
    reverse_entity.write('trigger_entity\t%s\n'%each)

###### Drug entity
drug_entity = {}
drugorder = 0
for ID in drugs :
    for drug in drugs[ID] :
        if drug.lower() in herbs or drug.lower() in ref_without :
            break
        drug_entity[drug.lower()] = 'drug_entity_%s'%drugorder
        try :
            reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drug.lower()))
        except :
            drugencode = str(drug.lower()).encode('utf8')
            reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drugencode))
    drugorder+=1

###### Herb entity 
herb_entity = {}
herborder = 0
for herb in herbs :
    herb_entity[herb.lower()] = 'herb_entity_%s'%herborder
    reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herb.lower()))
    for synonym in herbs[herb] :
        herb_entity[synonym.lower()] = 'herb_entity_%s'%herborder
        try :
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,synonym.lower()))
        except :
            herbencode = str(synonym.lower()).encode('utf8')
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herbencode))
    herborder+=1
    
###### Adverse effect entity
effect_entity = {}
effectorder = 0
for each in adverse_effect :
    effect_entity[each.lower()] = 'EF_entity_%s'%effectorder
    try :
        reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,each.lower()))
    except :
        effectencode = str(each.lower()).encode('utf8')
        reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effectencode))
    effectorder+=1
    
###### standard ref without effect entity 
for herb in ref_without :
    if herb.lower() not in herb_entity :
        herb_entity[herb.lower()] = 'herb_entity_%s'%herborder
        try :
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herb.lower()))
        except :
            herbencode = str(herb.lower()).encode('utf8')
            reverse_entity.write('herb_entity_%s\t%s\n'%(herborder,herbencode))
        herborder+=1
    for drug in ref_without[herb] :
        if drug.lower() not in drug_entity :
            drug_entity[drug.lower()] = 'drug_entity_%s'%drugorder
            try :
                reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drug.lower()))
            except :
                drugencode = str(drug.lower()).encode('utf8')
                reverse_entity.write('drug_entity_%s\t%s\n'%(drugorder,drugencode))
            drugorder+=1
            
###### standard ref with effect
''''''
#for row in ref_effect :
#    effect = row.split('\t')[2]
#    ###
#    if effect.lower() not in effect_entity :
#        effect_entity[effect.lower()] = 'EF_entity_%s'%effectorder
#        try :
#            reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effect.lower()))
#        except :
#            effectencode = str(effect.lower()).encode('utf8')
#            reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effectencode))
#        effectorder+=1
#    ###    
#    if len(row.split('\t')) == 4 :
#        effect1 = row.split('\t')[3]
#        if effect1.lower() not in effect_entity :
#            effect_entity[effect1.lower()] = 'EF_entity_%s'%effectorder
#            try :
#                reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effect1.lower()))
#            except :
#                effectencode = str(effect1.lower()).encode('utf8')
#                reverse_entity.write('EF_entity_%s\t%s\n'%(effectorder,effectencode))
#            effectorder+=1
''''''
######
reverse_entity.close()

In [62]:
###### remove '\n' from text and merge text
def rmv_n(textdict) :
    begin = time.time()
    text = ''
    print('merge text')
    count = 0
    for each in textdict :
        if len(textdict[each].split('\n')) != 1 :
            new = ''.join(textdict[each].split('\n'))
            text+=new
        else:
            text+= textdict[each]
        if count % 1000 == 0 :
            print('%s / %s'%(count,len(textdict)))
        count+=1
    text = text.lower()
    print(time.time() - begin)
    print('merge text done')
    return text

In [89]:
###### remove '\n' from text and merge text
def rmv_n(textdict) :
    begin = time.time()
    text = ''
    print('merge text')

    #text = ''.join(''.join(operator.itemgetter(*list(textdict))(textdict)).split('\n')).lower()
    
    text = ''.join(''.join([textdict[x] for x in list(textdict)]).split('\n')).lower()
    
    print(time.time() - begin)
    print('merge text done')
    return text

In [90]:
###### Entity replacement
def entity_replace(text,replaced_list) :
    ###
    begin = time.time()
    ###
    count = 0
    print('herb')
    for herb in herb_entity :
        if herb in text :
            text = text.replace(herb,' %s '%herb_entity[herb])
            #re.sub(herb,herb_entity[herb],text)
            if herb not in replaced_list :
                entity_replaced.write('%s\t%s\n'%(herb_entity[herb],herb))
                replaced_list.append(herb)
        if count % 1000 == 0 :
            print('%s / %s'%(count,len(herb_entity)))
        count+=1
    print('herb done')
    
    ###
    drug_entity_single = {}
    count = 0
    print('drug')
    for drug in drug_entity :
        if drug in text :
            text = text.replace(drug,' %s '%drug_entity[drug])
            #re.sub(drug,drug_entity[drug],text)
            if drug not in replaced_list :
                entity_replaced.write('%s\t%s\n'%(drug_entity[drug],drug))
                replaced_list.append(drug)
        #else:
            #drug_entity_single[drug] = drug_entity[drug]
        if count % 1000 == 0 :
            print('%s / %s'%(count,len(drug_entity)))
        count+=1
    print('drug done')
    ###
    print('effect')
    count = 0
    for effect in effect_entity :
        if effect in text :
            text = text.replace(effect,' %s '%effect_entity[effect])
            #re.sub(effect,effect_entity[effect],text)
            if effect not in replaced_list :
                entity_replaced.write('%s\t%s\n'%(effect_entity[effect],effect))
                replaced_list.append(effect)
        if count % 1000 == 0 :
            print('%s / %s'%(count,len(drug_entity)))
        count+=1
    print('effect done')
    ###
    for trig in trigger_entity:
        if trig in text:
            text = text.replace(trig,' %s '%trigger_entity[trig])
            #re.sub(trig,trigger_entity[trig],text)
            if trig not in replaced_list :
                entity_replaced.write('%s\t%s\n'%(trigger_entity[trig],trig))
                replaced_list.append(effect)
    print('trigger done')
    print(time.time() - begin)
    print('replace entities done')
    return text,replaced_list

In [91]:
######## word processing and word removal
def word_processing(text,track) :
    begin = time.time()
    count = 0

    ###### Create pos_tag accept list and stop words list and punctuations
    stop_words = list(set(stopwords.words('english')))
    pos_tag_adj = ['JJ','JJR','JJS']
    pos_tag_noun = ['NN','NNS','NNP','NNPS']
    punctuations = string.punctuation
    
    ###### Split text into words
    words = nltk.pos_tag(word_tokenize(text))
    text = ''
    total = len(words)
    print('Processing %s words'%total)
    #######
    sentences_full = []
    sentences_noun = []
    sentences_entity = []
    sentences_without_adj = []
    ########
    sentence_full = []
    sentence_noun = []
    sentence_entity = []
    sentence_without_adj = []
    ###### Processing
    for word in words :
        #######
        if word[0] != '.' : 
            ###### Full sentence
            if word[0] not in stop_words and word[0] not in punctuations :
                sentence_full.append(word[0])
                ###### Entity
                if 'entity' in word[0] :
                    sentence_entity.append(word[0])
                ###### Pos tagging
                if word[1] in pos_tag_noun or 'entity' in word[0] :
                    sentence_noun.append(word[0])
                if word[1] not in pos_tag_adj or 'entity' in word[0] :
                    sentence_without_adj.append(word[0])
        ###### Check point
        else : 
            ##### Save
            sentences_full.append(sentence_full)
            sentences_noun.append(sentence_noun)
            sentences_entity.append(sentence_entity)
            sentences_without_adj.append(sentence_without_adj)
            ##### Reset
            sentence_full = []
            sentence_noun = []
            sentence_entity = []
            sentence_without_adj = []
        
        ######
        if count % 100000 == 0 :
            print('Processed %s words'%count)
        count+=1
    
    ##### 
    print(time.time() - begin)
    print('Word processed done')
    
    ##### Only Noun
    outdir = '..//data//clean_data//only_noun'
    outfile = open(os.path.join(outdir,os.path.basename('processed_sentences_%s.json'%track)),'w')
    outfile.write(json.dumps(sentences_noun))
    outfile.close()
    sentences_noun = []
    ##### Only Entity
    outdir = '..//data//clean_data//only_entity'
    outfile = open(os.path.join(outdir,os.path.basename('processed_sentences_%s.json'%track)),'w')
    outfile.write(json.dumps(sentences_entity))
    outfile.close()
    sentences_entity = []
    ##### Without Adjective
    outdir = '..//data//clean_data//without_adjective'
    outfile = open(os.path.join(outdir,os.path.basename('processed_sentences_%s.json'%track)),'w')
    outfile.write(json.dumps(sentences_without_adj))
    outfile.close()
    sentences_without_adj = []
    ##### Full sentence
    outdir = '..//data//clean_data//full_sentence'
    outfile = open(os.path.join(outdir,os.path.basename('processed_sentences_%s.json'%track)),'w')
    outfile.write(json.dumps(sentences_full))
    outfile.close()
    sentences_full = []

In [54]:
######## word processing and word removal
def word_processing(text) :
    begin = time.time()
    count = 0

    ###### Create pos_tag accept list and stop words list 
    stop_words = list(set(stopwords.words('english')))
    pos_tag_accept = ['NN','NNS','NNP','NNPS','VB','VBD','VBG','VBN','VBP','VBZ']

    ###### Create entity list 
    reverse_entity = open('..//data/clean_data/entity_total.csv').read().split('\n')[:-1]
    entity_list = []
    for row in reverse_entity :
        entity_list.append(row.split('\t')[0])

    entity_list = list(set(entity_list))

    ###### Split text into sentences
    sentences = TextBlob(text).sentences
    total = len(sentences)
    print('Processing %s sentences'%total)
    sentences_processed = []

    ###### Processing
    for sentence in sentences :          
        words = word_tokenize(str(sentence)) ##### Split sentence into words 
        #####
        if len(set(words).intersection(entity_list)) > 2 : ##### Choose sentence contain more than 2 entities
            for word in words :
                if word in stop_words : ##### Remove stop words
                    words.pop(words.index(word))
                elif len(word) == 1 : ##### Remove punctuation marks
                    words.pop(words.index(word))    
            ###
            tags = nltk.pos_tag(words)
            for tag in tags :
                if tag[0] in entity_list : ##### Save entities
                    pass      
                elif tag[1] not in pos_tag_accept : ##### Remove words which are not noun,verb, and adjective 
                    words.pop(words.index(tag[0]))
            ###
            if len(words) > 3 : ###### Save processed sentence
                sentences_processed.append(words)
        ###
        if count % 10000 == 0 :
            print('Processed %s sentences'%count)
        count+=1

    ##### 
    print(time.time() - begin)
    print('Word processed done')
    return sentences_processed


In [92]:
########### Make outdir
os.system('mkdir "..//data//clean_data//only_noun"')
os.system('mkdir "..//data//clean_data//only_entity"')
os.system('mkdir "..//data//clean_data//without_adjective"')
os.system('mkdir "..//data//clean_data//full_sentence"')
########### Text source
text_list = os.listdir('..//data/preprocessing/text_extract/')
#text_list = ['text_extract_100.json']
count = 0
track = 0
replaced_list = []
textdict = []
for file in text_list :
    #####
    textdict = json.load(open('..//data//preprocessing//text_extract//%s'%file))
    #####
    print('Processing loop %s'%track)
    text = rmv_n(textdict)
    textdict = []
    #####
    print('step 1')
    (text,replaced_list) = entity_replace(text,replaced_list)
    #####
    print('step 2')
    sentences_processed = word_processing(text,track)
    text = ''
    #####
    ##### 
    track+=1

#########


#####
entity_replaced.close()

Processing loop 0
merge text
0.27300572395324707
merge text done
step 1
herb
0 / 30820
1000 / 30820
2000 / 30820
3000 / 30820
4000 / 30820
5000 / 30820
6000 / 30820
7000 / 30820
8000 / 30820
9000 / 30820
10000 / 30820
11000 / 30820
12000 / 30820
13000 / 30820
14000 / 30820
15000 / 30820
16000 / 30820
17000 / 30820
18000 / 30820
19000 / 30820
20000 / 30820
21000 / 30820
22000 / 30820
23000 / 30820
24000 / 30820
25000 / 30820
26000 / 30820
27000 / 30820
28000 / 30820
29000 / 30820
30000 / 30820
herb done
drug
0 / 25254
1000 / 25254
2000 / 25254
3000 / 25254
4000 / 25254
5000 / 25254
6000 / 25254
7000 / 25254
8000 / 25254
9000 / 25254
10000 / 25254
11000 / 25254
12000 / 25254
13000 / 25254
14000 / 25254
15000 / 25254
16000 / 25254
17000 / 25254
18000 / 25254
19000 / 25254
20000 / 25254
21000 / 25254
22000 / 25254
23000 / 25254
24000 / 25254
25000 / 25254
drug done
effect
0 / 25254
1000 / 25254
2000 / 25254
3000 / 25254
4000 / 25254
5000 / 25254
6000 / 25254
7000 / 25254
8000 / 25254
9000 

In [123]:
########### Text source
#text_list = os.listdir('..//data/preprocessing/text_extract/')
text_list = ['text_extract_100.json']
count = 0
track = 0
replaced_list = []
textdict = []
for file in text_list :
    if count == 0 :
        textdict = json.load(open('..//data//preprocessing//text_extract//%s'%file))
        count+=1
    elif count < 8 :
        textdict.update(json.load(open('..//data//preprocessing//text_extract//%s'%file)))
        count+=1
    elif count == 8 :
        textdict.update(json.load(open('..//data//preprocessing//text_extract//%s'%file)))
        count = 0
        #####
        print('Processing loop %s'%track)
        text = rmv_n(textdict)
        textdict = []
        #####
        print('step 1')
        (text,replaced_list) = entity_replace(text,replaced_list)
        #####
        print('step 2')
        sentences_processed = word_processing(text)
        text = ''
        #####
        ##### List of processed sentences
        outfile = open(os.path.join(outdir,os.path.basename('processed_sentences_%s.json'%track)),'w')
        outfile.write(json.dumps(sentences_processed))
        outfile.close()
        track+=1
        sentences_processed = []
        
#########
print('Processing loop %s'%track)
text = rmv_n(textdict)
textdict = []
#####
print('step 1')
(text,replaced_list) = entity_replace(text,replaced_list)
#####
print('step 2')
sentences_processed = word_processing(text)
text = ''
#####
##### List of processed sentences
outfile = open(os.path.join(outdir,os.path.basename('processed_sentences_%s.json'%track)),'w')
outfile.write(json.dumps(sentences_processed))
outfile.close()

#####
entity_replaced.close()

Processing loop 0
merge text
0.1980113983154297
merge text done
step 1
herb
0 / 30820
1000 / 30820
2000 / 30820
3000 / 30820
4000 / 30820
5000 / 30820
6000 / 30820


KeyboardInterrupt: 